In [1]:
import csv
import sqlite3
import requests
import io
from pprint import pprint
from tabulate import tabulate

# Body Fat - Get

The data set we use consists of 252 men with estimates of the percentage of body fat determined by underwater
weighing, biologic data, and various body circumference measurements. 

    1. Density determined from underwater weighing (gm/cm^3)
    2. Percent body fat from Siri's (1956) equation
    3. Age (years)
    4. Weight (lbs)
    5. Height (inches)
    6. Neck circumference (cm)
    7. Chest circumference (cm)
    8. Abdomen 2 circumference (cm)
    9. Hip circumference (cm)
    10. Thigh circumference (cm)
    11. Knee circumference (cm)
    12. Ankle circumference (cm)
    13. Biceps (extended) circumference (cm)
    14. Forearm circumference (cm)
    15. Wrist circumference (cm)
  
The data were generously supplied by Dr. A. Garth Fisher who gave permission to freely distribute the data and use for non-commercial purposes.

## Step 1.

Open the csv file and dig around.
What we're looking for is the basic entities, how their structure might vary, the formatting of data, and anything else that might cause problems for us.

In [2]:
list_ = []

url = 'http://staff.pubhealth.ku.dk/~tag/Teaching/share/data/Bodyfat.csv'
r = requests.get(url)
buff = io.StringIO(r.text)
dr = csv.DictReader(buff)

for row in dr:
    list_.append(row)

When poking around CSV files, it's better to look at types and keys first:

In [3]:
type(list_)

list

In [4]:
list_[0]

{'Density': '1.0708',
 'bodyfat': '12.3',
 'Age': '23',
 'Weight': '154.25',
 'Height': '67.75',
 'Neck': '36.2',
 'Chest': '93.1',
 'Abdomen': '85.2',
 'Hip': '94.5',
 'Thigh': '59',
 'Knee': '37.3',
 'Ankle': '21.9',
 'Biceps': '32',
 'Forearm': '27.4',
 'Wrist': '17.1'}

So we have a list of dictionaries, with each item corresponding to each row of data in the list.

In [5]:
len(list_)

252

Our list is 252 observations long, meaning we have 252 samples in our csv file. We can put it in a convenience variable and then look at a few.

In [6]:
pprint(list_[0:2], compact=True)

[{'Abdomen': '85.2',
  'Age': '23',
  'Ankle': '21.9',
  'Biceps': '32',
  'Chest': '93.1',
  'Density': '1.0708',
  'Forearm': '27.4',
  'Height': '67.75',
  'Hip': '94.5',
  'Knee': '37.3',
  'Neck': '36.2',
  'Thigh': '59',
  'Weight': '154.25',
  'Wrist': '17.1',
  'bodyfat': '12.3'},
 {'Abdomen': '83',
  'Age': '22',
  'Ankle': '23.4',
  'Biceps': '30.5',
  'Chest': '93.6',
  'Density': '1.0853',
  'Forearm': '28.9',
  'Height': '72.25',
  'Hip': '98.7',
  'Knee': '37.3',
  'Neck': '38.5',
  'Thigh': '58.7',
  'Weight': '173.25',
  'Wrist': '18.2',
  'bodyfat': '6.1'}]


Here we can see a number of things:

1. Each observation has a 15 variables.
2. The format for every value is a 1 decimal place float, except for weight and height which have two decimal places, density which has 4, and age which is an integer.

The layout of the table seems straightforward, however we should account for differences in measurements.  Earlier we noticed that some measurements were not in the same scale. For example, height is in inches but all the circumference measurements are in centimeters. Similarly, density is in gm/$cm^3$, but weight is in pounds. We should bare this in mind when we perform our EDA and modeling.

Lastly, the dataset already provides the body fat as determined by Siri's equation. In this exercise, however, we will be using Brozek's equation to approximate body fat as it has been shown to be more accurate for [all age groups](https://www.researchgate.net/profile/Jorge-Mota-8/publication/47813165_Accuracy_of_Siri_and_Brozek_equations_in_the_percent_body_fat_estimation_in_older_adults/links/0046351557ebd6fe68000000/Accuracy-of-Siri-and-Brozek-equations-in-the-percent-body-fat-estimation-in-older-adults.pdf). Therefore, we will add another row in our dataset that presents body fat % calculated by Brozek's equation. 

Our resulting table:
* BodyFat
    * id (Integer, Primary Key)
    * Density (Numeric)
    * Body Fat Siri (Numeric)
    * Body Fat Brozeks (Numeric)
    * Age (Integer)
    * Height (Numeric)
    * Weight (Numeric)
    * Neck (Numeric)
    * Chest (Numeric)
    * Abdomen (Numeric)
    * Hip (Numeric)
    * Thigh (Numeric)
    * Knee (Numeric)
    * Ankle (Numeric)
    * Biceps (Numeric)
    * Forearm (Numeric)
    * Wrist (Numeric)

## Step 2.

Put the data in the database.

In [7]:
!rm bodyfat.db

In [8]:
!sqlite3 bodyfat.db < bodyfat.sql

Establish a connection with the database

In [9]:
conn = sqlite3.connect('bodyfat.db')
cur = conn.cursor()

Next let's set up the helper function which will convert the density to body fat via Brozek's equation.

In [10]:
def brozeks(D):
    """
    Converts body density to body fat percentage via Brozek's equation
    
    """
    
    bodyfat =  ((4.570/float(D) - 4.142) * 100)
    bodyfat = float(str(round(bodyfat, 1)))
    return (bodyfat)

We will next upload our data into the datawarehouse.

In [11]:
for row in list_:
    #find all the variables
    density = row["Density"]
    bodyfat_siri = row["bodyfat"]
    bodyfat_brozek = brozeks(row["Density"])
    age = row["Age"]
    height = row["Height"]
    weight = row["Weight"]
    neck = row["Neck"]
    chest = row["Chest"]
    abdomen = row["Abdomen"]
    hip = row["Hip"]
    thigh = row["Thigh"]
    knee = row["Knee"]
    ankle = row["Ankle"]
    biceps = row["Biceps"]
    forearm = row["Forearm"]
    wrist = row["Wrist"]
        
    #insert into table
    sql = f"""INSERT INTO Body_Fat (BodyFat_siri, Density, Age, Height, 
        Weight, Neck, Chest, Abdomen, Hip, Thigh, BodyFat_brozek,
        Knee, Ankle, Biceps, Forearm, Wrist)
            VALUES ('{bodyfat_siri}', '{density}', '{age}', '{height}',
            '{weight}', '{neck}', '{chest}', '{abdomen}', '{hip}',
            '{thigh}', '{bodyfat_brozek}','{knee}', '{ankle}', '{biceps}',
            '{forearm}','{wrist}');
        """
    cur.execute(sql)
conn.commit()

Now let's check to see our information loaded. We should have 252 rows.

In [12]:
print("Number of observations:", list(cur.execute("SELECT COUNT(*) FROM Body_Fat"))[0][0])

Number of observations: 252


Great, we've loaded all 252 rows. Let's do a quick query to make sure everythin loaded correctly. 

In [13]:
query = """
SELECT BodyFat_brozek
FROM Body_Fat
WHERE BodyFat_brozek > 30
"""
result1 = list(cur.execute(query))
tabulate(result1, headers=["BodyFat"], tablefmt='html')

BodyFat
31.1
38.2
33.8
31.3
33.1
31.7
30.4
30.8
30.4
31


Our datawarehouse is ready for EDA and modeling.

### Source: ###

The data were generously supplied by Dr. A. Garth Fisher who gave permission to freely distribute the data and use for non-commercial purposes.

Roger W. Johnson\
Department of Mathematics & Computer Science\
South Dakota School of Mines & Technology\
501 East St. Joseph Street\
Rapid City, SD 57701

email address: rwjohnso@silver.sdsmt.edu\
web address: http://silver.sdsmt.edu/~rwjohnso